## Download specific versions and source code locally

In [18]:
# torch==1.10.2
!pip install transformers==4.16.2 datasets==1.17.0 tokenizers==0.11.6 wandb==0.12.14 seqeval

     |████████████████████████████████| 43 kB 629 kB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=6a35f90def1fbeb559277abe5afc03ec187025900522fde92e1d29ff220e8f96
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval


In [2]:
!nvidia-smi

Tue Apr 12 09:44:41 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8    33W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
%cd /content
!rm -r ntu_nlp_al
!git clone -b kretov/adding_tagging https://github.com/DevKretov/ntu_nlp_al

/content
rm: cannot remove 'ntu_nlp_al': No such file or directory
Cloning into 'ntu_nlp_al'...
remote: Enumerating objects: 143, done.
remote: Counting objects: 100% (143/143), done.
remote: Compressing objects: 100% (96/96), done.
remote: Total 143 (delta 76), reused 105 (delta 43), pack-reused 0
Receiving objects: 100% (143/143), 56.57 MiB | 10.55 MiB/s, done.
Resolving deltas: 100% (76/76), done.
Checking out files: 100% (24/24), done.


In [5]:

%cd ntu_nlp_al/

/content/ntu_nlp_al


## Initialisation

In [6]:
import torch
from transformers import get_scheduler
from datasets import list_metrics, load_metric

from active_learning_trainer import ALTrainer
from transformers import AutoTokenizer
from dataset import Dataset, TokenClassificationDataset
from model import Model


import datetime
import wandb


2022-04-12 09:45:07,276 git.cmd DEBUG:Popen(['git', 'version'], cwd=/content/ntu_nlp_al, universal_newlines=False, shell=None, istream=None)
2022-04-12 09:45:07,310 git.cmd DEBUG:Popen(['git', 'version'], cwd=/content/ntu_nlp_al, universal_newlines=False, shell=None, istream=None)
2022-04-12 09:45:08,533 urllib3.connectionpool DEBUG:Starting new HTTPS connection (1): huggingface.co:443
2022-04-12 09:45:09,309 urllib3.connectionpool DEBUG:https://huggingface.co:443 "HEAD /bert-base-multilingual-uncased/resolve/main/config.json HTTP/1.1" 200 0
2022-04-12 09:45:09,319 filelock DEBUG:Attempting to acquire lock 140106392212368 on /root/.cache/huggingface/transformers/af4e101d208f361f141144dca21e9c4148aaf0e85441c2e335743d10829c6cad.d63adade93e44e64bedd306ec82ffd33eedabaf0ff08aabe581acaa48616a508.lock
2022-04-12 09:45:09,330 filelock DEBUG:Lock 140106392212368 acquired on /root/.cache/huggingface/transformers/af4e101d208f361f141144dca21e9c4148aaf0e85441c2e335743d10829c6cad.d63adade93e44e64bed

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

2022-04-12 09:45:10,160 filelock DEBUG:Attempting to release lock 140106392212368 on /root/.cache/huggingface/transformers/af4e101d208f361f141144dca21e9c4148aaf0e85441c2e335743d10829c6cad.d63adade93e44e64bedd306ec82ffd33eedabaf0ff08aabe581acaa48616a508.lock
2022-04-12 09:45:10,161 filelock DEBUG:Lock 140106392212368 released on /root/.cache/huggingface/transformers/af4e101d208f361f141144dca21e9c4148aaf0e85441c2e335743d10829c6cad.d63adade93e44e64bedd306ec82ffd33eedabaf0ff08aabe581acaa48616a508.lock


In [11]:
parameters = dict()
parameters['use_gpu'] = True

parameters['weights_and_biases_on'] = True
parameters['weights_and_biases_key'] = '5e5e00356042a33b5cb271399b8d05c9c9d6ded8'
# TODO: run name based on timestamp
current_timestamp = str(datetime.datetime.now()).split('.')[0]

# TODO: implement it
parameters['weights_and_biases_save_predictions'] = False

parameters['pretrained_model_name'] = 'prajjwal1/bert-small' #'distilbert-base-uncased'


# parameters['train_dataset_file_path'] = 'data/imdb/train_IMDB.csv'
# parameters['val_dataset_file_path'] = 'data/imdb/test_IMDB.csv'
# parameters['test_dataset_file_path'] = 'data/imdb/test_IMDB.csv'

parameters['dataset_from_datasets_hub'] = True
parameters['dataset_from_datasets_hub_name'] = 'conll2003'
parameters['train_dataset_file_path'] = 'data/news/train.csv'
parameters['val_dataset_file_path'] = 'data/news/val.csv'
parameters['test_dataset_file_path'] = 'data/news/test.csv'
parameters['dataset_file_delimiter'] = ','

### FIRST IS FOR TAGGING
parameters['dataset_text_column_name'] =  'tokens' #'text'
parameters['dataset_label_column_name'] = 'ner_tags'#'airline_sentiment'

### THIS IS FOR CLASSIFICATION
### FIRST OF ALL, LOOK AT COLUMNS IN DATASET CSV!!! THEN WRITE HERE!
#parameters['dataset_text_column_name'] = 'text_cleaned'  # 'text'
#parameters['dataset_label_column_name'] = 'label_reduced'  # 'airline_sentiment'

# TODO: implement this with CrossEntropyLoss
parameters['loss'] = 'cross_entropy'
parameters['loss_weighted'] = False

parameters['class_imbalance_reweight'] = True
parameters['train_batch_size'] = 32
parameters['val_batch_size'] = 64
parameters['test_batch_size'] = 64
parameters['epochs'] = 5
# parameters['finetuned_model_type'] = 'classification'
parameters['finetuned_model_type'] = 'tagging'
model_type = parameters['finetuned_model_type']


parameters['al_iterations'] = 50
parameters['init_dataset_size'] = 32
parameters['add_dataset_size'] = 32
### HERE CAN BE least_confidence OR badge OR random OR kmeans OR entropy
parameters['al_strategy'] = 'least_confidence' #'least_confidence'
# IF YOU WANT TO TRAIN ON THE WHOLE MODEL
parameters['full_train'] = False

parameters['debug'] = False

In [12]:
device = 'cpu'
if parameters['use_gpu']:
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

parameters['weights_and_biases_run_name'] = f'{current_timestamp}_run_{model_type}_{device}'
if parameters['debug']:
    parameters['weights_and_biases_run_name'] = 'DEBUG_' + parameters['weights_and_biases_run_name']
print(f'Device set to {device}!')

Device set to cuda:0!


## Preparing tokenizer, dataset, model

In [13]:
tokenizer = AutoTokenizer.from_pretrained(parameters['pretrained_model_name'])

dataset_obj = None
if parameters['finetuned_model_type'] == 'classification':
    dataset_obj = Dataset(tokenizer)
elif parameters['finetuned_model_type'] == 'tagging':
    dataset_obj = TokenClassificationDataset(tokenizer)
else:
    model_type = parameters['finetuned_model_type']
    raise NotImplementedError(f'Type {model_type} not supported yet!')



2022-04-12 09:48:19,385 urllib3.connectionpool DEBUG:Starting new HTTPS connection (1): huggingface.co:443
2022-04-12 09:48:20,145 urllib3.connectionpool DEBUG:https://huggingface.co:443 "HEAD /prajjwal1/bert-small/resolve/main/tokenizer_config.json HTTP/1.1" 404 0
2022-04-12 09:48:20,152 urllib3.connectionpool DEBUG:Starting new HTTPS connection (1): huggingface.co:443
2022-04-12 09:48:20,901 urllib3.connectionpool DEBUG:https://huggingface.co:443 "HEAD /prajjwal1/bert-small/resolve/main/config.json HTTP/1.1" 200 0
2022-04-12 09:48:20,909 urllib3.connectionpool DEBUG:Starting new HTTPS connection (1): huggingface.co:443
2022-04-12 09:48:21,659 urllib3.connectionpool DEBUG:https://huggingface.co:443 "HEAD /prajjwal1/bert-small/resolve/main/tokenizer_config.json HTTP/1.1" 404 0
2022-04-12 09:48:21,663 urllib3.connectionpool DEBUG:Starting new HTTPS connection (1): huggingface.co:443
2022-04-12 09:48:22,413 urllib3.connectionpool DEBUG:https://huggingface.co:443 "HEAD /prajjwal1/bert-sma

In [14]:
if parameters['dataset_from_datasets_hub']:
    dataset_name = parameters['dataset_from_datasets_hub_name']
    dataset_obj.load_hosted_dataset(dataset_name)
else:
    data_files = {
        'train': [parameters['train_dataset_file_path']],
        'val': [parameters['val_dataset_file_path']],
        'test': [parameters['test_dataset_file_path']]
    }

    dataset_obj.load_csv_dataset(
        data_files,
        delimiter=parameters['dataset_file_delimiter']
    )

dataset_obj.truncate_dataset('train', 10000)
dataset_obj.truncate_dataset('val', 1000)
dataset_obj.truncate_dataset('test', 10000)

dataset_obj.prepare_dataset(parameters['dataset_label_column_name'], parameters['dataset_text_column_name'], )
# dataset_obj.prepare_labels(parameters['dataset_label_column_name'])
# dataset_obj.encode_dataset(parameters['dataset_text_column_name'])

# From NLP Example: https://docs.wandb.ai/examples
wandb_table = None
if parameters['weights_and_biases_on'] and \
    parameters['weights_and_biases_save_predictions']:


    categories = sorted(list(dataset_obj.get_all_categories().items()), key=lambda key: key[1])
    categories_names = [_tuple[0] for _tuple in categories]
    wandb_table_columns = ['seq_i', 'text', 'true_label', 'prediction'] + categories_names
    wandb_table = wandb.Table(columns=wandb_table_columns)


print(f'Categories: {dataset_obj.get_all_categories()}')
num_labels = dataset_obj.get_num_categories()

2022-04-12 09:48:27,086 urllib3.connectionpool DEBUG:Starting new HTTPS connection (1): s3.amazonaws.com:443
2022-04-12 09:48:27,846 urllib3.connectionpool DEBUG:https://s3.amazonaws.com:443 "HEAD /datasets.huggingface.co/datasets/datasets/conll2003/conll2003.py HTTP/1.1" 200 0
2022-04-12 09:48:27,854 urllib3.connectionpool DEBUG:Starting new HTTPS connection (1): raw.githubusercontent.com:443
2022-04-12 09:48:28,247 urllib3.connectionpool DEBUG:https://raw.githubusercontent.com:443 "HEAD /huggingface/datasets/master/datasets/conll2003/conll2003.py HTTP/1.1" 200 0
2022-04-12 09:48:28,255 urllib3.connectionpool DEBUG:Starting new HTTPS connection (1): raw.githubusercontent.com:443
2022-04-12 09:48:28,306 urllib3.connectionpool DEBUG:https://raw.githubusercontent.com:443 "GET /huggingface/datasets/master/datasets/conll2003/conll2003.py HTTP/1.1" 200 2581


Downloading:   0%|          | 0.00/2.58k [00:00<?, ?B/s]

2022-04-12 09:48:28,363 urllib3.connectionpool DEBUG:Starting new HTTPS connection (1): raw.githubusercontent.com:443
2022-04-12 09:48:28,763 urllib3.connectionpool DEBUG:https://raw.githubusercontent.com:443 "HEAD /huggingface/datasets/master/datasets/conll2003/dataset_infos.json HTTP/1.1" 200 0
2022-04-12 09:48:28,769 urllib3.connectionpool DEBUG:Starting new HTTPS connection (1): raw.githubusercontent.com:443
2022-04-12 09:48:28,820 urllib3.connectionpool DEBUG:https://raw.githubusercontent.com:443 "GET /huggingface/datasets/master/datasets/conll2003/dataset_infos.json HTTP/1.1" 200 1618


Downloading:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

2022-04-12 09:48:28,917 urllib3.connectionpool DEBUG:Starting new HTTPS connection (1): storage.googleapis.com:443


...

  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Categories: {'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6, 'B-MISC': 7, 'I-MISC': 8}


In [15]:
model = Model(
        parameters['pretrained_model_name'],
        model_type=parameters['finetuned_model_type'],
        num_labels=num_labels
    )

2022-04-12 09:48:42,753 urllib3.connectionpool DEBUG:Starting new HTTPS connection (1): huggingface.co:443
2022-04-12 09:48:43,506 urllib3.connectionpool DEBUG:https://huggingface.co:443 "HEAD /prajjwal1/bert-small/resolve/main/config.json HTTP/1.1" 200 0
2022-04-12 09:48:43,514 urllib3.connectionpool DEBUG:Starting new HTTPS connection (1): huggingface.co:443
2022-04-12 09:48:44,264 urllib3.connectionpool DEBUG:https://huggingface.co:443 "HEAD /prajjwal1/bert-small/resolve/main/pytorch_model.bin HTTP/1.1" 302 0
2022-04-12 09:48:44,269 filelock DEBUG:Attempting to acquire lock 140106394482896 on /root/.cache/huggingface/transformers/facfdb1638fdec899406e0efd5c2c43ae4bbafcb45dd15f68df1f2378e3e70fb.59547972ec02ba39d4ea413c843f1638e8f90e118a4334ae5d626bf7524ac597.lock
2022-04-12 09:48:44,271 filelock DEBUG:Lock 140106394482896 acquired on /root/.cache/huggingface/transformers/facfdb1638fdec899406e0efd5c2c43ae4bbafcb45dd15f68df1f2378e3e70fb.59547972ec02ba39d4ea413c843f1638e8f90e118a4334ae5

Downloading:   0%|          | 0.00/111M [00:00<?, ?B/s]

2022-04-12 09:48:48,055 filelock DEBUG:Attempting to release lock 140106394482896 on /root/.cache/huggingface/transformers/facfdb1638fdec899406e0efd5c2c43ae4bbafcb45dd15f68df1f2378e3e70fb.59547972ec02ba39d4ea413c843f1638e8f90e118a4334ae5d626bf7524ac597.lock
2022-04-12 09:48:48,056 filelock DEBUG:Lock 140106394482896 released on /root/.cache/huggingface/transformers/facfdb1638fdec899406e0efd5c2c43ae4bbafcb45dd15f68df1f2378e3e70fb.59547972ec02ba39d4ea413c843f1638e8f90e118a4334ae5d626bf7524ac597.lock
Some weights of the model checkpoint at prajjwal1/bert-small were not used when initializing BertForTokenClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTok

### Connect to W&B environment

In [16]:
if parameters['weights_and_biases_on']:
    wandb.login(key='5e5e00356042a33b5cb271399b8d05c9c9d6ded8')
    wandb.init(
        name=parameters['weights_and_biases_run_name'],
        project='ntu_al',
        reinit=True
    )

    wandb.config.update(parameters)
    wandb.watch(model.model)

2022-04-12 09:48:52,039 urllib3.connectionpool DEBUG:Starting new HTTP connection (1): 172.28.0.2:9000
2022-04-12 09:48:52,047 urllib3.connectionpool DEBUG:http://172.28.0.2:9000 "GET /api/sessions?token= HTTP/1.1" 200 420
2022-04-12 09:48:52,059 urllib3.connectionpool DEBUG:Starting new HTTPS connection (1): api.wandb.ai:443
2022-04-12 09:48:52,249 urllib3.connectionpool DEBUG:https://api.wandb.ai:443 "POST /graphql HTTP/1.1" 200 None
2022-04-12 09:48:52,255 urllib3.connectionpool DEBUG:Starting new HTTPS connection (1): api.wandb.ai:443
2022-04-12 09:48:52,502 urllib3.connectionpool DEBUG:https://api.wandb.ai:443 "POST /graphql HTTP/1.1" 200 None
wandb: W&B API key is configured (use `wandb login --relogin` to force relogin)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to you

### Init trainer and combine everything

In [19]:
trainer = ALTrainer(
    wandb_on=parameters['weights_and_biases_on'],
    imbalanced_training=parameters['class_imbalance_reweight'],
    model_type=parameters['finetuned_model_type'],
    wandb_table=wandb_table
)
trainer.set_model(model)
# TODO: add strategy
trainer.set_strategy(None)
trainer.set_dataset(dataset_obj)
dataset_obj.prepare_dataloaders(
    train_batch_size=parameters['train_batch_size'],
    val_batch_size=parameters['val_batch_size'],
    test_batch_size=parameters['test_batch_size'],
)

optimizer = torch.optim.AdamW(model.model.parameters(), lr=5e-5)
trainer.set_optimizer(optimizer)

num_training_steps = parameters['epochs'] * trainer.get_training_steps_num()

lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)
trainer.set_lr_scheduler(lr_scheduler)
trainer.set_device(device)

trainer.add_evaluation_metric(load_metric('accuracy'))

if parameters['finetuned_model_type'] == 'tagging':
    trainer.add_evaluation_metric(load_metric('seqeval'))
else:
    trainer.add_evaluation_metric(load_metric('f1'))
    trainer.add_evaluation_metric(load_metric('precision'))
    trainer.add_evaluation_metric(load_metric('recall'))

2022-04-12 09:49:44,086 urllib3.connectionpool DEBUG:Starting new HTTPS connection (1): s3.amazonaws.com:443
2022-04-12 09:49:44,873 urllib3.connectionpool DEBUG:https://s3.amazonaws.com:443 "HEAD /datasets.huggingface.co/datasets/metrics/accuracy/accuracy.py HTTP/1.1" 200 0
2022-04-12 09:49:44,879 urllib3.connectionpool DEBUG:Starting new HTTPS connection (1): raw.githubusercontent.com:443
2022-04-12 09:49:45,294 urllib3.connectionpool DEBUG:https://raw.githubusercontent.com:443 "HEAD /huggingface/datasets/1.17.0/metrics/accuracy/accuracy.py HTTP/1.1" 200 0
2022-04-12 09:49:45,320 urllib3.connectionpool DEBUG:Starting new HTTPS connection (1): s3.amazonaws.com:443
2022-04-12 09:49:46,093 urllib3.connectionpool DEBUG:https://s3.amazonaws.com:443 "HEAD /datasets.huggingface.co/datasets/metrics/seqeval/seqeval.py HTTP/1.1" 200 0
2022-04-12 09:49:46,107 urllib3.connectionpool DEBUG:Starting new HTTPS connection (1): raw.githubusercontent.com:443
2022-04-12 09:49:46,170 urllib3.connectionp

## Train

In [ ]:
if parameters['full_train']:
    trainer.full_train(
        train_epochs=parameters['epochs'],
        train_batch_size=parameters['train_batch_size'],
        val_batch_size=parameters['val_batch_size'],
        test_batch_size=parameters['test_batch_size'],
        debug=parameters['debug']
    )

trainer.al_train(
    al_iterations=parameters['al_iterations'],
    init_dataset_size=parameters['init_dataset_size'],
    add_dataset_size=parameters['add_dataset_size'],
    train_epochs=parameters['epochs'],
    strategy=parameters['al_strategy'],
    train_batch_size=parameters['train_batch_size'],
    val_batch_size=parameters['val_batch_size'],
    test_batch_size=parameters['test_batch_size'],
    debug=parameters['debug']
)

2022-04-12 09:49:46,526 root INFO:Training initialized!


  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

2022-04-12 09:49:50,614 root DEBUG:AL train dataset length: 32, rest dataset length: 9968
2022-04-12 09:49:50,631 root INFO:Training is run on 1 batches!
2022-04-12 09:49:50,633 root INFO:Evaluation is run on 16 batches!
2022-04-12 09:49:50,634 root INFO:Testing is run on 54 batches!
2022-04-12 09:49:50,637 root INFO:

2022-04-12 09:49:50,641 root INFO:AL Least confidence strategy applied!
2022-04-12 09:49:50,643 root INFO:
2022-04-12 09:49:50,646 root INFO:
AL iteration   1/50
2022-04-12 09:49:50,651 urllib3.connectionpool DEBUG:Starting new HTTPS connection (1): huggingface.co:443
2022-04-12 09:49:51,404 urllib3.connectionpool DEBUG:https://huggingface.co:443 "HEAD /prajjwal1/bert-small/resolve/main/config.json HTTP/1.1" 200 0
2022-04-12 09:49:51,427 urllib3.connectionpool DEBUG:Starting new HTTPS connection (1): huggingface.co:443
2022-04-12 09:49:52,207 urllib3.connectionpool DEBUG:https://huggingface.co:443 "HEAD /prajjwal1/bert-small/resolve/main/pytorch_model.bin HTTP/1.1" 302 0

Training mean loss: 2.1200218200683594: 100%|██████████| 1/1 [00:00<00:00,  3.34it/s]

...

2022-04-12 09:51:16,568 root INFO:Epoch finished. Evaluation:
2022-04-12 09:51:16,570 root DEBUG:Is model on CUDA - cuda:0
2022-04-12 09:51:16,577 root INFO:Running Eval mode...


Eval mean loss: 0.7398164495825768: 100%|██████████| 16/16 [00:02<00:00,  6.24it/s]

2022-04-12 09:51:19,149 root INFO:Test set evaluation:
2022-04-12 09:51:19,154 root DEBUG:Is model on CUDA - cuda:0
2022-04-12 09:51:19,157 root INFO:Running Test mode...



Test mean loss: 0.6876846115898203: 100%|██████████| 54/54 [00:08<00:00,  7.27it/s]

2022-04-12 09:51:27,382 root INFO:
Metrics, confusion matrix
2022-04-12 09:51:28,482 root INFO:
{conf_matrix}


In [ ]:
wandb.finish()